In [9]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

from sklearn.neighbors import KNeighborsClassifier

warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

class processutil:
    def _str2class(s):
        if s in globals() and isinstance(globals()[s], type):
                return globals()[s]
        if isinstance(eval(s), type):
            return eval(s)
        if callable(eval(s)):
            return eval(s)
        return None
    

Using TensorFlow backend.


In [3]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [4]:
df_train = pd.read_pickle(f'{folder_path}/df_train3.gzde', compression='gzip')#.iloc[:80000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test3.gzde', compression='gzip')#.iloc[:50000,:]
df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(-999)
df_test = df_test.replace([np.inf, -np.inf], np.nan).fillna(-999)

In [6]:
sorted_columns=[
 'PCA_C_1',
 'C12',
 'PCA_C_0',
 'V317',
 'D10',
 'C13_fq_enc',
 'card1_TransactionAmt_mean',
 'uid3_TransactionAmt_mean',
 'lgb-single_V317',
 'lgb-single_C1',
 'PCA_C_2',
 'C13',
 'M4_target_mean',
 'uid_TransactionAmt_mean',
 'uid3_TransactionAmt_std',
 'id_06',

 'extensive-_PCA_V_27',
 'D1',
 'eda-and-mo_C1',
 'PCA_V_27',
 'PCA_V_28',
 'uid2_TransactionAmt_mean',
 'eda-and-mo_V317',
 'card1',
 
 'PCA_V_13',
 'card2_TransactionAmt_mean',
 'P_emaildomain__C2',
 'PCA_V_2',
 'uid3_fq_enc',
 'card1__card5',
 'device_name',
 'Card_ID',
 'D2_fq_enc',
 'addr1__card1',
 'TransactionAmt_to_std_card_id',
 'extensive-_PCA_V_4',
 'P_emaildomain_fq_enc',
 'almost-com_TransactionAmt',
 'addr1_fq_enc',
 'D2',
 'PCA_V_1',
 'DeviceInfo_device',
 'TransactionAmt_to_mean_card4',
 'eda-and-mo_TransactionAmt_to_std_card1',
 'ieee-gb-2-_DeviceInfo',
 'C11',
 'PCA_V_4',
 'DeviceInfo__P_emaildomain',
 'TransactionAmt_to_std_card4',
 'card2_fq_enc',
 'uid_DT_D',
 'D4_fq_enc',
 'PCA_V_10',
 'extensive-_id_02',
 'extensive-_PCA_V_24',
 'card5__P_emaildomain',
 'TransactionAmt_to_mean_addr1',
 'C4',
 'uid',
 'TransactionAmt_to_std_card1',
 'uid2_fq_enc',
 'V42',
 'card1_TransactionAmt_std',
 'uid_TransactionAmt_std',
 'card5',
 'TransactionAmt_decimal',
 'V307',
 'PCA_V_25',
 'V294',
 'feature-en_TransactionAmt_to_std_card1',
 'DeviceInfo_fq_enc',
 'PCA_V_22',
 'max_last',
 'card1_fq_enc',
 'uid_DT_M',
 'PCA_V_5',
 'PCA_V_34',
 'D4',
 'ieee-gb-2-_M5',
 'extensive-_PCA_V_22',
 'card1_count_full',
 'PCA_V_6',
 'extensive-_PCA_V_13',
 'uid2',
 '_Hours',
 'eda-and-mo_card1',
 'PCA_V_15',
 'ieee-gb-2-_M4',
 'card2__dist1',
 'DeviceInfo',
 'PCA_V_24',
 'extensive-_PCA_V_25',
 'card5_TransactionAmt_mean',
 'V224',
 'P_emaildomain_prefix',
 '_Second',
 'id_01',
 'id_02_to_std_card1',
 'card2_TransactionAmt_std',
 'P_emaildomain_suffix',
 'extensive-_PCA_V_14',
 'card6',
 'eda-and-mo_id_19',
 'eda-and-mo_TransactionAmt_to_mean_card1',
 'uid_fq_enc',
 'extensive-_PCA_V_19',
 'PCA_V_14',
 'card2',
 'PCA_V_26',
 'V310',
 'card5_count_full',
 'eda-and-mo_V307',
 'card6_count_full',
 'Transaction_hour',
 'V313',
 'PCA_V_12',
 'lgb-single_V307',
 'C6',
 'uid3',
 'R_emaildomain_1',
 'lgb-single_V314',
 '_Minutes',
 'uid_DT_W',
 'version_id_31',
 'D15_to_mean_card4',
 'almost-com_id_02',
 'PCA_V_17',
 'uid2_TransactionAmt_std',
 'ieee-gb-2-_card1',
 'PCA_V_3',
 'R_emaildomain_fq_enc',
 'eda-and-mo_id_02_to_std_card1',
 'almost-com_card2',
 'PCA_D_6',
 'id_01_count_dist',
 'eda-and-mo_id_20',
 'eda-and-mo_TransactionAmt_to_mean_card4',
 'card2_count_full',
 'V248',
 'R_emaildomain',
 'C14',
 'lgb-single_V207',
 'V84',
 'eda-and-mo_V268',
 'V78',
 'lgb-single_V267',
 'PCA_V_19',
 'V137',
 'extensive-_PCA_V_15',
 'eda-and-mo_addr1',
 'card3_count_full',
 'V87',
 'DT_hour',
 'V135',
 'std_last',
 'TransactionAmt_to_std_addr1',
 'V312',
 'extensive-_PCA_V_28',
 'card5_TransactionAmt_std',
 'eda-and-mo_V310',
 'V295',
 'V203',
 'V207',
 'D11__DeviceInfo',
 'feature-en_TransactionAmt_to_mean_card1',
 'eda-and-mo_D15_to_mean_card1',
 'first_value_card1',
 'PCA_V_18',
 'PCA_V_31',
 'PCA_V_29',
 'extensive-_PCA_V_8',
 'eda-and-mo_V264',
 'V258',
 'Transaction_day_of_week',
 'eda-and-mo_V203',
 'V261',
 'card3_TransactionAmt_mean',
 'D15',
 'lgb-single_D8',
 'lgb-single_C11',
 'D1_fq_enc',
 'V285',

 'V268',
 'extensive-_id_06',
 'DT_D_total',
 'PCA_D_4',
 'D8',
 'id_19',
 'lgb-single_D9',
 'extensive-_PCA_V_5',
 'feature-en_TransactionAmt_to_mean_card4',
 'PCA_V_30',
 'V267',
 'ieee-gb-2-_R_emaildomain',
 'TransactionAmt_to_mean_card1',
 'eda-and-mo_V312',
 'lgb-single_V310',
 'V221',
 'ProductCD',
 'feature-en_P_emaildomain_bin',
 'extensive-_PCA_V_10',
 'PCA_V_9',
 'extensive-_id_01',
 'C14_fq_enc',
 'extensive-_DeviceInfo',
 'id_05',
 'make_hour_feature',
 'V283',
 'V130',
 'device_version',
 'eda-and-mo_V313',
 'V314',
 'lgb-single_V313',
 'V178',
 'TransactionAmt_log1p',
 'lgb-single_V128',
 'DeviceInfo_version_fq_enc',
 'almost-com_M5',
 'R_emaildomain_prefix',
 'C10_fq_enc',
 'V265',
 'id_36',
 'extensive-_PCA_V_12',
 'PCA_V_16',
 'V259',
 'almost-com_P_emaildomain',
 'D15_to_std_card4',
 'eda-and-mo_card2',
 'card4_count_full',
 'id_14',
 'lgb-single_V208',
 'ieee-gb-2-_P_emaildomain',
 'id_02_to_mean_card4',
 'extensive-_PCA_V_11',
 'eda-and-mo_D15_to_std_card1',
 'extensive-_id_38',
 'D11',
 'PCA_V_32',
 'V291',
 'D5_fq_enc',
 'lgb-single_V131',
 'lgb-single_V130',
 'V54',
 'V227',
 'V237',
 'V229',
 'V43',
 'eda-and-mo_V278',
 'V204',
 'eda-and-mo_V314',
 'D3',
 'V222',
 'V205',
 'eda-and-mo_V332',
 'V186',
 'V216',
 'V215',
 'V214',
 'eda-and-mo_V215',
 'make_day_feature',
 'V234',
 'V183',
 'lgb-single_V278',
 'D15_to_mean_addr1',
 
 'V136',
 'V271',
 'V39',
 'lgb-single_V271',
 'V126',
 'V272',
 'V273',
 'lgb-single_V268',
 'V275',
 'id_33_0',
 'lgb-single_V216',
 'V109',
 'V106',
 'eda-and-mo_V271',
 'lgb-single_V272',
 'V169',
 'V254',
 'V182',
 'D15_to_std_addr1',
 'V86',
 'eda-and-mo_D9',
 'eda-and-mo_D15_to_std_card4',
 'V157',
 'V104',
 'lgb-single_V306',
 'lgb-single_V215',
 'C11_fq_enc',
 'extensive-_PCA_V_17',
 'eda-and-mo_card5',
 'V61',
 'eda-and-mo_D15_to_mean_card4',
 'eda-and-mo_id_02_to_std_card4',
 'extensive-_PCA_V_21',
 'almost-com_M4',
 'eda-and-mo_V308',
 'V315',
 'count_cluster',
 'eda-and-mo_TransactionAmt_to_std_card4',
 'D8_fq_enc',
 'almost-com_id_09',
 'D3_fq_enc',
 'eda-and-mo_C2',
 'PCA_V_33',
 'V133',
 'V257',
 'V131',
 'C2_fq_enc',
 'PCA_V_20',
 'eda-and-mo_V267',
 'V44',
 'V180',
 'V177',
 'V208',
 'V230',
 'V95',
 'V91',
 'ieee-gb-2-_ProductCD',
 'lgb-single_V265',
 'lgb-single_V277',
 'eda-and-mo_id_02_to_mean_card4',
 'V217',
 'eda-and-mo_V315',
 'id_02',
 'ieee-gb-2-_card6',
 'feature-en_TransactionAmt_to_std_card4',
 'eda-and-mo_dist1',
 'almost-com_id_01',
 'eda-and-mo_C11',
 'V287',
 'V17',
 'lgb-single_C2',
 'id_02_to_mean_card1',
 'C12_fq_enc',
 'id_02__id_20',
 'V67',
 'V13',
 'id_30_version',
 'clusters_D',
 'C9_fq_enc',
 'id_35',
 'clusters_V',
 'V26',
 'ieee-gb-2-_M6',
 'lgb-single_V126',
 'V37',
 'extensive-_id_30',
 'ProductCD_target_mean',
 'id_02_to_mean_addr1',
 'extensive-_id_18',
 'PCA_D_5',
 'extensive-_id_13',
 'almost-com_dist1',
 'almost-com_id_11',
 'id_02_to_std_card4',
 'V165',
 'extensive-_PCA_V_23',
 'D15_to_std_addr2',
 'extensive-_PCA_V_18',
 'PCA_V_8',
 'id_02__D8',
 'extensive-_PCA_V_16',
 'V12',
 'dist1',
 'V29',
 'almost-com_M6',
 'V45',
 'V279',
 'almost-com_ProductCD',
 'V308',
 'lgb-single_V264',
 'PCA_D_1',
 'dist2_fq_enc',
 'extensive-_R_emaildomain_bin',
 'TransactionAmt',
 'mean_last',
 'first_value_addr1',
 'extensive-_PCA_V_20',
 'almost-com_card4',
 'V60',
 'V306',
 'V309',
 'V59',
 'V27',
 'eda-and-mo_V210',
 'V150',
 'P_emaildomain_1',
 'extensive-_id_03',
 'extensive-_DeviceType',
 'ieee-gb-2-_M3',
 'id_33_count_dist',
 'PCA_D_7',
 'V255',
 'PCA_D_2',
 'PCA_V_0',
 'M_sum',
 'extensive-_id_19',
 'D15_to_std_card1',
 'C10',
 'extensive-_id_32',
 'V264',
 'V25',
 'lgb-single_dist1',
 'id_33',
 'ieee-gb-2-_id_33',
 'extensive-_id_34',
 'eda-and-mo_V127',
 'V75',
 'V301',
 'V76',
 'extensive-_PCA_V_6',
 'id_31_device_fq_enc',
 'M8',
 'id_31_device',
 'C8',
 'P_emaildomain',
 'TransactionAmt_Log',
 'lgb-single_TransactionAmt',
 'eda-and-mo_id_13',
 'V35',
 'M3',
 'M2',
 'lgb-single_V315',
 'id_30',
 'V108',
 'id_37',
 'eda-and-mo_D15_to_mean_addr1',
 'eda-and-mo_V130',
 'eda-and-mo_V265',
 'eda-and-mo_V277',
 'V196',
 'V160',
 'V318',
 'V159',
 'V155',
 'V153',
 'V132',
 'V129',
 'id_15',
 'V149',
 'eda-and-mo_id_14',
 'C1',
 'extensive-_id_11',
 'eda-and-mo_D8',
 'lgb-single_V203',
 'V164',
 'extensive-_id_26',
 'V23',
 'ieee-gb-2-_id_30',
 'extensive-_id_22',
 'V316',
 'P_emaildomain_2',
 'V278',
 'V53',
 'V331',
 'ieee-gb-2-_M9',
 'card2__id_20',
 'V62',
 'V70',
 'card3_TransactionAmt_std',
 'card4',
 'almost-com_id_05',
 'D15_to_mean_addr2',
 'extensive-_id_31',
 'V16',
 'id_30_version_fq_enc',
 'TransactionAmt_to_mean_card_id',
 'extensive-_PCA_V_26',
 'lgb-single_V274',
 'id_02_to_std_addr1',
 'V152',
 'V158',
 'extensive-_id_36',
 'had_id',
 'extensive-_id_37',
 'V147',
 'V156',
 'feature-en_P_emaildomain_suffix',
 'feature-en_R_emaildomain_suffix',
 'V15',
 'V151',
 'V146',
 'V154',
 'V48',
 'V145',
 'id_12',
 'id_25',
 'id_24',
 'id_23',
 'id_22',
 'id_21',
 'V124',
 'id_18',
 'id_16',
 'V125',
 'id_11',
 'V144',
 'id_10',
 'id_09',
 'id_08',
 'id_07',
 'V134',
 'id_03',
 'V138',
 'V141',
 'V142',
 'V14',
 'extensive-_id_27',
 'extensive-_id_35',
 'V161',
 'V174',
 'V175',
 'V176',
 'V179',
 'V18',
 'V181',
 'V184',
 'V185',
 'V187',
 'V188',
 'V189',
 'V19',
 'V190',
 'V191',
 'V192',
 'V193',
 'V194',
 'V195',
 'V197',
 'V199',
 'V2',
 'V49',
 'extensive-_P_emaildomain_suffix',
 'V172',
 'extensive-_id_17',
 'V162',
 'extensive-_id_29',
 'extensive-_id_28',
 'id_27',
 'extensive-_id_25',
 'extensive-_id_24',
 'extensive-_id_23',
 'extensive-_id_21',
 'V163',
 'extensive-_id_16',
 'extensive-_R_emaildomain_suffix',
 'extensive-_id_15',
 'V166',
 'extensive-_id_12',
 'extensive-_id_09',
 'extensive-_id_08',
 'extensive-_id_07',
 'V167',
 'V168',
 'V171',
 'id_26',
 'V121',
 'id_28',
 'id_29',
 'DT_day',
 'M1',
 'lgb-single_V276',
 'lgb-single_V273',
 'lgb-single_V270',
 'OS_id_30',
 'lgb-single_V266',
 'PCA_V_21',
 'lgb-single_V214',
 'lgb-single_V213',
 'lgb-single_V212',
 'lgb-single_V210',
 'lgb-single_V209',
 'lgb-single_V206',
 'lgb-single_V205',
 'lgb-single_V204',
 'lgb-single_V202',
 'lgb-single_V150',
 'P_emaildomain_bin',
 'R_emaildomain_2',
 'R_emaildomain_bin',
 'lgb-single_V332',
 'lgb-single_V335',
 'lgb-single_V336',
 'screen_width',
 'version_id_30',
 'C3',
 'C3_fq_enc',
 'C4_fq_enc',
 'C7',
 'C7_fq_enc',
 'D12',
 'D13',
 'D14',
 'screen_height',
 'DT_W_total',
 'D5',
 'D6',
 'D7',
 'D7_fq_enc',
 'DT_D',
 'DT_M',
 'DT_M_total',
 'DT_W',
 'lgb-single_id_11',
 'TransactionAmt_check',
 'TransactionDT',
 'TransactionID',
 'id_33_fq_enc',
 'ieee-gb-2-_M1',
 'ieee-gb-2-_D9',
 'id_38',
 'V114',
 'id_36_count_full',
 'id_36_count_dist',
 'V115',
 'V116',
 'id_34',
 'V117',
 'V111',
 'V118',
 'V119',
 'id_32',
 'V120',
 'V200',
 'V122',
 'id_30_device_fq_enc',
 'id_30_device',
 'V123',
 'ieee-gb-2-_M2',
 'V110',
 'V1',
 'ieee-gb-2-_id_23',
 'V10',
 'ieee-gb-2-_id_38',
 'ieee-gb-2-_id_36',
 'ieee-gb-2-_id_35',
 'ieee-gb-2-_id_34',
 'V100',
 'ieee-gb-2-_id_29',
 'ieee-gb-2-_id_28',
 'ieee-gb-2-_id_27',
 'ieee-gb-2-_id_16',
 'V11',
 'ieee-gb-2-_id_12',
 'V101',
 'ieee-gb-2-_card4',
 'V102',
 'V103',
 'V105',
 'V107',
 'ieee-gb-2-_M8',
 'ieee-gb-2-_M7',
 'email_check',
 'V173',
 'V298',
 'V325',
 'browser_id_31',
 'V319',
 'almost-com_id_07',
 'almost-com_id_03',
 'V32',
 'V320',
 'V321',
 'V322',
 'V323',
 'V324',
 'V326',
 'V31',
 'V327',
 'V328',
 'V329',
 'almost-com_M1',
 'addr2_fq_enc',
 'addr2',
 'V33',
 'V330',
 'V332',
 'V333',
 'V311',
 'V305',
 'V269',
 'V288',
 'V276',
 'V277',
 'V28',
 'V280',
 'clusters_C',
 'V281',
 'V282',
 'card5_fq_enc',
 'V284',
 'V286',
 'V289',
 'V304',
 'V290',
 'V292',
 'V296',
 'V201',
 'V299',
 'V3',
 'V30',
 'V300',
 'V302',
 'V303',
 '_Days',
 'V99',
 'V98',
 'V40',
 'V73',
 'V72',
 'V71',
 'V7',
 'V69',
 'V68',
 'V38',
 'V65',
 'V64',
 'V4',
 'V41',
 'V97',
 'V46',
 'V58',
 'V57',
 'V56',
 'V55',
 'V47',
 'V52',
 'V51',
 'V50',
 'V5',
 'V74',
 'V36',
 'V34',
 'V77',
 'V96',
 'V334',
 'V94',
 'V93',
 'V92',
 'V335',
 'V90',
 'V9',
 'V89',
 'V88',
 'V336',
 'V337',
 'V85',
 'V338',
 'V83',
 'V82',
 'V81',
 'V80',
 'V8',
 'V79',
 'V339',
 'V270',
 'V6',
 'V266',
 'V232',
 'V233',
 'V235',
 'eda-and-mo_V276',
 'eda-and-mo_V275',
 'eda-and-mo_V273',
 'eda-and-mo_V272',
 'V236',
 'eda-and-mo_V269',
 'V238',
 'V239',
 'eda-and-mo_V266',
 'V24',
 'V240',
 'eda-and-mo_V263',
 'eda-and-mo_V216',
 'V241',
 'eda-and-mo_V214',
 'eda-and-mo_V213',
 'eda-and-mo_V212',
 'eda-and-mo_V211',
 'V242',
 'eda-and-mo_V306',
 'V231',
 'eda-and-mo_V206',
 'V228',
 'V202',
 'V206',
 'eda-and-mo_id_11',
 'V209',
 'V21',
 'V210',
 'V211',
 'V212',
 'eda-and-mo_card3',
 'V213',
 'V218',
 'eda-and-mo_addr2',
 'V219',
 'eda-and-mo_V336',
 'eda-and-mo_V335',
 'eda-and-mo_V334',
 'V22',
 'V220',
 'V223',
 'V225',
 'V226',
 'eda-and-mo_V209',
 'eda-and-mo_V270',
 'eda-and-mo_V205',
 'V256',
 'V250',
 'V247',
 'eda-and-mo_TransactionAmt',
 'V246',
 'eda-and-mo_V126',
 'V251',
 'eda-and-mo_V128',
 'V244',
 'V249',
 'eda-and-mo_V131',
 'V253',
 'V252',
 'eda-and-mo_V150',
 'eda-and-mo_V204',
 'V260',
 'V262',
 'eda-and-mo_C7',
 'V243',
 'V263',
 'eda-and-mo_V202',
 'extensive-_id_33',
 'C6_fq_enc',
 'extensive-_TransactionAmt',
 'extensive-_id_20',
 'extensive-_PCA_V_9',
 'eda-and-mo_V207',
 'dist2',
 'V293',
 'M5',
 'eda-and-mo_id_17',
 'V297',
 'lgb-single_V312',
 'id_33_1',
 'card3_fq_enc',
 'almost-com_addr2',
 'dist1_fq_enc',
 'V128',
 'lgb-single_V275',
 'V20',
 'V66',
 'V139',
 'ieee-gb-2-_id_15',
 'id_04',
 'V148',
 'V170',
 'M6',
 'feature-en_R_emaildomain_bin',
 'id_31',
 'id_30_fq_enc',
 'D9',
 'PCA_D_3',
 'ieee-gb-2-_id_37',
 '_Weekdays',
 'eda-and-mo_V208',
 'C9',
 'lgb-single_V127',
 'M4',
 'eda-and-mo_id_02_to_mean_card1',
 'PCA_V_23',
 'lgb-single_C7',
 'extensive-_id_04',
 'ieee-gb-2-_id_31',
 'V127',
 'V143',
 'D15_to_mean_card1',
 'extensive-_PCA_V_29',
 'almost-com_M7',
 'extensive-_id_14',
 'id_13',
 'V63',
 'extensive-_id_05',
 'V140',
 'M7',
 'almost-com_addr1',
 'V112',
 'id_17',
 'V198',
 'V113',
 'eda-and-mo_id_32',
 'DT_day_week',
 'V245',
 'eda-and-mo_V274',
 'eda-and-mo_C12',
 'extensive-_P_emaildomain_bin',
 'lgb-single_V263',
 'R_emaildomain_suffix',
 'card3',
 'lgb-single_C12',
 'DeviceType',
 'addr1',
 'almost-com_R_emaildomain',
 'D6_fq_enc',
 'M_na',
 'V274',
 'lgb-single_V308',
 'PCA_D_0',
 'M9',
 'id_31_count_dist',
 'C8_fq_enc',
 'lgb-single_C8',
 'eda-and-mo_C8',
 'DeviceInfo_version',
 'PCA_V_11',
 'extensive-_id_10',
 'C5_fq_enc',
 'C5',
 'min_last',
 'id_20',
 'C1_fq_enc',
 'extensive-_PCA_V_7',
 'PCA_V_7',
 'C2',
 'DeviceInfo_device_fq_enc']

In [7]:
len(sorted_columns)

925

In [10]:
def process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
            
#         trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
#         val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
#         clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
        
        clf = KNeighborsClassifier(n_neighbors=10)
        clf.fit(X, y)
        
        y_trn_pred = clf.predict(X.iloc[trn_idx].values)
        y_val_pred = clf.predict(X.iloc[test_idx].values)
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict(X_test.values)
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        
        
#         aucs.append(clf.best_score['valid_1']['auc'])
#         his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [13]:
folds = TimeSeriesSplit(n_splits=5)
# folds = KFold(n_splits=8, shuffle=True, random_state=42)

In [11]:
df_train = df_train[sorted_columns[:208] + ['isFraud', 'TransactionID', 'TransactionDT']]
df_test = df_test[sorted_columns[:208] + ['TransactionID', 'TransactionDT']]

In [14]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, sorted_columns[:208], is_output_feature_importance=0, verbose=2)

Training on fold 1
Fold 1 finished in 0:39:04.474354
Training on fold 2
Fold 2 finished in 1:06:59.571053
Training on fold 3
Fold 3 finished in 1:35:26.647098
Training on fold 4
Fold 4 finished in 2:04:40.569169
Training on fold 5
Fold 5 finished in 2:32:49.186422
------------------------------
Training has finished.
Total training time is 7:59:01.288248
Mean AUC: 0.6713951760478766 0.6377773365150097
------------------------------
